In [10]:
import analytics as an
from analytics.materials.ions import Cation, Anion
from analytics.materials.electrolytes import Electrolyte
from analytics.materials.solvents import Solvent
from analytics.experiment_modelling.cyclic_voltammetry import CyclicVoltammogram
import pandas as pd
import plotly.express as px
from scipy import integrate
import matplotlib.pyplot as plt
import numpy as np

In [11]:
my_cation = Cation('Li+')
my_anion = Anion('Cl-')
my_solvent = Solvent('water')

my_electrolyte = Electrolyte(solvent=my_solvent, cation=my_cation, anion=my_anion, concentrations={my_cation: 0.1, my_anion: 0.1})


In [12]:
my_cv = CyclicVoltammogram.from_benelogic("../test_trajectories/cyclic_voltammetry/benelogic1.txt", electrolyte=my_electrolyte, keep_cycle_1 = True)

In [13]:
my_cv.show_current_potential(template='plotly_dark')

In [14]:
my_cv.show_current_time(template='plotly_dark')

In [15]:
my_cv.get_charge_passed()

,cycle,redox,anodic_charge,cathodic_charge
0,1,oxidation,0.002489,-0.000059
1,1,reduction,0.000004,-0.003265
2,2,oxidation,0.002498,-0.000053
3,2,reduction,0.000004,-0.003260
4,3,oxidation,0.002502,-0.000050
5,3,reduction,0.000004,-0.003257
6,4,oxidation,0.002504,-0.000049
7,4,reduction,0.000004,-0.003254


In [16]:
direction = 'reduction'
current = 'positive'

def get_root(x1, x2, y1, y2):
    slope = (y2 - y1) / (x2 - x1)
    intercept = y1 - slope * x1
    root = -intercept / slope
    return root

if current == 'positive':
    current_data = data.query('current > 0')
elif current == 'negative':
    current_data = data.query('current < 0')

if direction == 'oxidation':
    int_current = current_data.query('redox == "oxidation"')['current'].to_numpy()
    int_time = current_data.query('redox == "oxidation"')['time'].to_numpy()
elif direction == 'reduction':
    int_current = current_data.query('redox == "reduction"')['current'].to_numpy()
    int_time = current_data.query('redox == "reduction"')['time'].to_numpy()

if direction == 'oxidation' and current == 'negative':
    max_index = data.query('redox == "oxidation" and current < 0').index.max() + 1
    y1 = int_current[-1]
    x1 = int_time[-1]
    y2 = data.query('index == @max_index')['current'].iloc[0]
    x2 = data.query('index == @max_index')['time'].iloc[0]
    root = get_root(x1, x2, y1, y2)
    int_current = np.append(int_current, 0)
    int_time = np.append(int_time, root)

elif direction == 'oxidation' and current == 'positive':
    min_index = data.query('redox == "oxidation" and current > 0').index.min() - 1
    y1 = int_current[0]
    x1 = int_time[0]
    y2 = data.query('index == @min_index')['current'].iloc[0]
    x2 = data.query('index == @min_index')['time'].iloc[0]
    root = get_root(x1, x2, y1, y2)
    int_current = np.append(0, int_current)
    int_time = np.append(root, int_time)

elif direction == 'reduction' and current == 'negative':
    min_index = data.query('redox == "reduction" and current < 0').index.min() - 1
    y1 = int_current[0]
    x1 = int_time[0]
    y2 = data.query('index == @min_index')['current'].iloc[0]
    x2 = data.query('index == @min_index')['time'].iloc[0]
    root = get_root(x1, x2, y1, y2)
    int_current = np.append(0, int_current)
    int_time = np.append(root, int_time)

elif direction == 'reduction' and current == 'positive':
    max_index = data.query('redox == "reduction" and current > 0').index.max() + 1
    y1 = int_current[-1]
    x1 = int_time[-1]
    y2 = data.query('index == @max_index')['current'].iloc[0]
    x2 = data.query('index == @max_index')['time'].iloc[0] 
    root = get_root(x1, x2, y1, y2)
    int_current = np.append(int_current, 0)
    int_time = np.append(int_time, root)

integral = integrate.simpson(int_current, int_time)

px.line(x = int_time, y = int_current, title='Cyclic Voltammogram', template='plotly_dark', width=1200, height=400, markers=True).show()
integral*1000


NameError: name 'data' is not defined

In [ ]:
# step 2 get the negative current and store in a new dataframe. Get the minimum and maximum indices of the negative current

neg_current = data.query('current < 0')

min_index = neg_current.index.min() - 1
max_index = neg_current.index.max() + 1

px.line(neg_current, x='time', y='current', title='Cyclic Voltammogram', color='redox', template='plotly_dark', width=600, height=400, markers=True).show()

print(f"the min index is {min_index} and the max index is {max_index}")
neg_current

the min index is 1073 and the max index is 1084


,potential,current,cycle,time,redox
1074,-0.499772,-0.000382,2,106.260997,oxidation
1075,-0.497185,-0.000141,2,106.311997,oxidation
1076,-0.492182,-0.000089,2,106.411997,oxidation
1077,-0.487175,-0.000072,2,106.511997,oxidation
1078,-0.482181,-0.000058,2,106.611997,oxidation
1079,-0.477182,-0.000047,2,106.711997,oxidation
1080,-0.471407,-0.000035,2,106.826997,oxidation
1081,-0.465650,-0.000023,2,106.941997,oxidation
1082,-0.460654,-0.000014,2,107.041997,oxidation
1083,-0.455658,-0.000006,2,107.141997,oxidation


In [ ]:
# step 3 check if index on the left side is the same cycle and redox direction. If it is then add the new point

if len(data.query('index == @max_index and cycle == @cycle_num and redox == @direction and current > 0')) > 0:
    
    current_to_int = neg_current['current'].to_numpy()
    time_to_int = neg_current['time'].to_numpy()

    x1 = neg_current['time'].iloc[-1]
    x2 = data.query('index == @max_index')['time'].iloc[0]
    y1 = neg_current['current'].iloc[-1]
    y2 = data.query('index == @max_index')['current'].iloc[0]

    slope = (y2 - y1) / (x2 - x1)
    intercept = y1 - slope * x1
    root = -intercept / slope

    current_to_int = np.append(current_to_int, 0)
    time_to_int = np.append(time_to_int, root)

    px.line(x=time_to_int, y=current_to_int, title='Cyclic Voltammogram', template='plotly_dark', width=600, height=400, markers=True).show()

    area = integrate.simpson(current_to_int, time_to_int)

area


-5.040958999159462e-05